# Importamos librerias - Creamos el dataframe de pericias

In [3]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)
df1

0      \n\nInforma a continuación: \n\nJ.T. 13 \nPER...
1      \n\nJUZG. 40 \n\nPERITO MEDICO PRESENTA INFOR...
2     Expediente Nº CNT 74803/2014 \n \nJUZGADO NACI...
3     JUZGADO NACIONAL DE TRABAJO 59       LAVALLE 1...
4      \n\n \n\nPERITO PRESENTA INFORME \n\n \n\n \n...
5                           JNT N°72 \n\nPERITO MEDI...
6     J. T.  Nº 20              Exp. 9231//2016     ...
7     PERITO MEDICO PRESENTA INFORME \n\nSr. Juez: \...
8     II.- HISTORIA CLINICA \n\nPERITO MEDICO PRESEN...
9     JNT nº 67, Sito en Calle Lavalle nº 1268, Ciud...
10    PERITO MEDICO: PRESENTA PERITACION MEDICA.- \n...
11     \n\nSR. JUEZ: \n\nJUZGADO NACIONAL DEL TRABAJ...
12    PRESENTA DICTAMEN PERICIAL \n\nSeñor Juez: \n\...
13    JUZG. NAC. DEL TRABAJO Nº 12 \n\n             ...
14    SE PRESENTA – HACE SABER: \n\nJuz T 51 \n\n \n...
15    PERITO MÉDICO PRESENTA INFORME PERICIAL\n\nSr....
16    PRESENTA INFORME PERICIAL MÉDICO. \n\nJ.N.T. 1...
17    PERITO MEDICO PRESENTA INFORME PERICIAL \n

## Limpieza de texto

In [7]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)


In [10]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

dfLimpio = df1.apply(limpiarTexto2)

'Informa a continuacion: \n\nJ.T. 13 \nPERITO MEDICO PRESENTA INFORME PERICIAL \n \nAutos: “BOIERO FRANCO IVAN c/SWISS MEDICAL ART S.A. s/Accidente - Ley Especial” \nExpediente N.º: 71399/2017 \n \nI-PROEMIO: \n \nSr. JUEZ NACIONAL: \n \nHORTAS MARIA ANDREA S. perito medico de oficio en los siguientes \nautos, con domicilio constituido en la calle Austria 1754 3º piso Dpto. 10 de esta \nCapital Z 172, tel. 4325-8802 domicilio electronico mashortas@hotmail.com, 27-\n16523394-3 en cumplimiento lo dispuesto por V.S. referente a los presentes autos \ny despues de haber examinado a quien dijo ser Boiero Franco Ivan quien se \ntomaron en el momento del examen el estado clinico actual se consideraron la \ncapacidad generica y en particular los puntos de pericia solicitados por las partes \ny/o V.S.: \n \n \n \nII-ANTECEDENTES DE INTERES MEDICO-LEGAL: \nFoja 3 Inicia la demanda \nFoja 6 vta. Hechos accidente in itinere refiere que el 20 de setiembre del 2017 siendo \nlas 5:10h se encontraba en

In [14]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    return text

dfLimpio = dfLimpio.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)
#dfLimpio[0]

# Busqueda de titulos

In [37]:
# Busqueda de titulos
# Se crea un nuevo dataframe de titulos según distintos criterios

# busca titulos que comienzan con numeros romanos
def buscarTitulosRomanos(text):
    # Expresiones regulares para encontrar los numeros romanos
    tituloNroRomano =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][\w\s]+[:\n]')
    titulosRomanosEncontrados = []

    for m in tituloNroRomano.finditer(text):
        titulosRomanosEncontrados.append(m.group())

    return titulosRomanosEncontrados

# titulosConNroRomano: lista que guarda los titulos que comienzan con numeros romanos   
titulosConNroRomano=[]
for expediente in dfLimpio:
    titulosConNroRomano.append(buscarTitulosRomanos(expediente))

pprint(titulosConNroRomano[0])

['I-PROEMIO:',
 'III-EXAMEN FISICO DEL ACTOR:',
 'III-1 PROSOGRAFIA:',
 'III-2 MOTIVO DE LA EVALUACION:',
 'III-3 DATOS PERSONALES:',
 'III-4 RELATO DE LOS HECHOS:',
 'V.A\n'
 'La articulacion es factible que por la inestabilidad el actor sufra otra '
 'lesion\n'
 'El actor deberia completar tratamiento kinesico a fin de fortificar los\n',
 'VII-INCAPACIDAD:',
 'IX-PETITORIO:']


In [30]:
# Busca titulos que comienzan con numeros latinos
def buscarTitulosLatinos(text):
    # Expresion regular para encontrar titulos que comienzan con numeros
    tituloNroLatino =re.compile(r'[1-9][-.)][\w\s]+[:\n]')
    titulosLatinosEncontrados = []

    for m in tituloNroLatino.finditer(text):
        titulosLatinosEncontrados.append(m.group())

    return titulosLatinosEncontrados

# titulosConNroLatino: lista que guarda los titulos que comienzan con numeros latinos
titulosConNroLatino=[]
for expediente in dfLimpio:
    titulosConNroLatino.append(buscarTitulosLatinos(expediente))
pprint(titulosConNroLatino[0])

['5-SEMIOLOGIA DE LAS AFECCIONES:',
 '2. TOBILLO:',
 '4. PSICOENTREVISTA:',
 '1.\n'
 'Las secuelas en el tobillo izquierdo que presenta el actor es consecuencia\n',
 '2.\n'
 'Estas conclusiones son en base al hecho que aun persiste las dificultades\n',
 '6.\n'
 'Las dolencias y dificultades para la movilidad que presenta el actor\n'
 'determina que presente dificultades intermedias para la realizacion de '
 'tareas\n',
 '7.\nQue a los efectos fiscales denuncio mi CUIT :',
 '1-De la parte actora\n1 Psicologicos\n',
 '1-De la parte demandada\n']


In [38]:
# Busca titulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresion regular para encontrar titulos en mayusculas.
    tituloMayusculas =re.compile(r'([1-9]|I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][A-Z -]+[:.\n]')
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los titulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))


#### Eliminacion de stopwords y lematizacion.

In [42]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

df1 = df1.apply(remove_stops)

print(df1[0])

AttributeError: module 'scipy' has no attribute '_lib'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

documento = df1.apply(str)  
# crear la transformación  
vectorizer = CountVectorizer() 
# tokenizar y construir el vocabulario
vectorizer.fit(documento) 
  
print("Vocabulary: ", vectorizer.vocabulary_) 
# codificador de documentos
vector = vectorizer.transform(documento) 

# resumir vector codificado
#print("El documento codificado es:") 
#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

In [ ]:
# r'.\n +'[0-9]+[-. ]+[a-zA-z ]+'

#.\n +'([vV][.-]+ [A-Za-z ]+)|([iI{0,3}][vV]+[.-]+ [A-Za-z ]+)